In [2]:
import pandas as pd
from datetime import datetime
from sklearn import preprocessing
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn import tree
import numpy as np 
import pandas as pd 
import os

**Training**

In [3]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/covid19-global-forecasting-week-5/train.csv
/kaggle/input/covid19-global-forecasting-week-5/submission.csv
/kaggle/input/covid19-global-forecasting-week-5/test.csv


In [4]:
path = '/kaggle/input/covid19-global-forecasting-week-5/'
train_data = pd.read_csv(path+"train.csv")
train_data

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
0,1,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-23,ConfirmedCases,0
1,2,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-23,Fatalities,0
2,3,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-24,ConfirmedCases,0
3,4,NaN,NaN,Afghanistan,27657145,0.583587,2020-01-24,Fatalities,0
4,5,NaN,NaN,Afghanistan,27657145,0.058359,2020-01-25,ConfirmedCases,0
...,...,...,...,...,...,...,...,...,...
969635,969636,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-08,Fatalities,0
969636,969637,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-09,ConfirmedCases,27
969637,969638,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-09,Fatalities,0
969638,969639,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-10,ConfirmedCases,6


In [5]:
train_data["County"].count()

880040

In [6]:
train_data["Province_State"].count()

917280

In [7]:
train_data["Country_Region"].count()

969640

In [8]:
train_data["Population"].count()

969640

In [9]:
train_data["Weight"].count()

969640

In [10]:
train_data["Date"].count()

969640

In [11]:
train_data["Target"].count()

969640

In [12]:
train_data.dropna(subset = ["County", "Province_State", "Country_Region", "Population", "Weight", "Date", "Target"], inplace=True)
train_data.reset_index(drop=True, inplace=True)
train_data

,Id,County,Province_State,Country_Region,Population,Weight,Date,Target,TargetValue
0,67761,Autauga,Alabama,US,55869,0.091485,2020-01-23,ConfirmedCases,0
1,67762,Autauga,Alabama,US,55869,0.914848,2020-01-23,Fatalities,0
2,67763,Autauga,Alabama,US,55869,0.091485,2020-01-24,ConfirmedCases,0
3,67764,Autauga,Alabama,US,55869,0.914848,2020-01-24,Fatalities,0
4,67765,Autauga,Alabama,US,55869,0.091485,2020-01-25,ConfirmedCases,0
...,...,...,...,...,...,...,...,...,...
880035,962636,Weston,Wyoming,US,6927,1.130796,2020-06-08,Fatalities,0
880036,962637,Weston,Wyoming,US,6927,0.113080,2020-06-09,ConfirmedCases,0
880037,962638,Weston,Wyoming,US,6927,1.130796,2020-06-09,Fatalities,0
880038,962639,Weston,Wyoming,US,6927,0.113080,2020-06-10,ConfirmedCases,0


In [13]:
train_attributes = train_data[["County", "Province_State", "Country_Region", "Population", "Weight", "Date"]]

In [14]:
le_train_county = LabelEncoder()
le_train_province_state = LabelEncoder()
le_train_country_region = LabelEncoder()
le_train_date = LabelEncoder()

train_attributes["County_n"] = le_train_county.fit_transform(train_attributes["County"])
train_attributes["Province_State_n"] = le_train_province_state.fit_transform(train_attributes["Province_State"])
train_attributes["Country_Region_n"] = le_train_country_region.fit_transform(train_attributes["Country_Region"])
train_attributes["Date_n"] = le_train_date.fit_transform(train_attributes["Date"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [15]:
train_attributes

,County,Province_State,Country_Region,Population,Weight,Date,County_n,Province_State_n,Country_Region_n,Date_n
0,Autauga,Alabama,US,55869,0.091485,2020-01-23,81,0,0,0
1,Autauga,Alabama,US,55869,0.914848,2020-01-23,81,0,0,0
2,Autauga,Alabama,US,55869,0.091485,2020-01-24,81,0,0,1
3,Autauga,Alabama,US,55869,0.914848,2020-01-24,81,0,0,1
4,Autauga,Alabama,US,55869,0.091485,2020-01-25,81,0,0,2
...,...,...,...,...,...,...,...,...,...,...
880035,Weston,Wyoming,US,6927,1.130796,2020-06-08,1763,50,0,137
880036,Weston,Wyoming,US,6927,0.113080,2020-06-09,1763,50,0,138
880037,Weston,Wyoming,US,6927,1.130796,2020-06-09,1763,50,0,138
880038,Weston,Wyoming,US,6927,0.113080,2020-06-10,1763,50,0,139


In [16]:
train_attributes_n = train_attributes.drop(["County", "Province_State", "Country_Region", "Date"], axis = "columns")
train_attributes_n

,Population,Weight,County_n,Province_State_n,Country_Region_n,Date_n
0,55869,0.091485,81,0,0,0
1,55869,0.914848,81,0,0,0
2,55869,0.091485,81,0,0,1
3,55869,0.914848,81,0,0,1
4,55869,0.091485,81,0,0,2
...,...,...,...,...,...,...
880035,6927,1.130796,1763,50,0,137
880036,6927,0.113080,1763,50,0,138
880037,6927,1.130796,1763,50,0,138
880038,6927,0.113080,1763,50,0,139


In [17]:
train_target_class = train_data[["Target"]]

In [18]:
le_target = LabelEncoder()
train_target_class["Target_n"] = le_target.fit_transform(train_target_class["Target"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [19]:
train_target_class

,Target,Target_n
0,ConfirmedCases,0
1,Fatalities,1
2,ConfirmedCases,0
3,Fatalities,1
4,ConfirmedCases,0
...,...,...
880035,Fatalities,1
880036,ConfirmedCases,0
880037,Fatalities,1
880038,ConfirmedCases,0


In [20]:
train_target_class_n = train_target_class.drop(["Target"], axis = "columns")
train_target_class_n

,Target_n
0,0
1,1
2,0
3,1
4,0
...,...
880035,1
880036,0
880037,1
880038,0


In [21]:
model = tree.DecisionTreeClassifier()
model.fit(train_attributes_n, train_target_class_n)

DecisionTreeClassifier()

In [22]:
model.score(train_attributes_n, train_target_class_n)

1.0

**Testing**

In [23]:
test_data = pd.read_csv(path+"test.csv")
test_data

,ForecastId,County,Province_State,Country_Region,Population,Weight,Date,Target
0,1,NaN,NaN,Afghanistan,27657145,0.058359,2020-04-27,ConfirmedCases
1,2,NaN,NaN,Afghanistan,27657145,0.583587,2020-04-27,Fatalities
2,3,NaN,NaN,Afghanistan,27657145,0.058359,2020-04-28,ConfirmedCases
3,4,NaN,NaN,Afghanistan,27657145,0.583587,2020-04-28,Fatalities
4,5,NaN,NaN,Afghanistan,27657145,0.058359,2020-04-29,ConfirmedCases
...,...,...,...,...,...,...,...,...
311665,311666,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-08,Fatalities
311666,311667,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-09,ConfirmedCases
311667,311668,NaN,NaN,Zimbabwe,14240168,0.607106,2020-06-09,Fatalities
311668,311669,NaN,NaN,Zimbabwe,14240168,0.060711,2020-06-10,ConfirmedCases


In [24]:
test_data.dropna(subset = ["County", "Province_State", "Country_Region", "Population", "Weight", "Date", "Target"], inplace=True)
test_data.reset_index(drop=True, inplace=True)
test_data

,ForecastId,County,Province_State,Country_Region,Population,Weight,Date,Target
0,21781,Autauga,Alabama,US,55869,0.091485,2020-04-27,ConfirmedCases
1,21782,Autauga,Alabama,US,55869,0.914848,2020-04-27,Fatalities
2,21783,Autauga,Alabama,US,55869,0.091485,2020-04-28,ConfirmedCases
3,21784,Autauga,Alabama,US,55869,0.914848,2020-04-28,Fatalities
4,21785,Autauga,Alabama,US,55869,0.091485,2020-04-29,ConfirmedCases
...,...,...,...,...,...,...,...,...
282865,309416,Weston,Wyoming,US,6927,1.130796,2020-06-08,Fatalities
282866,309417,Weston,Wyoming,US,6927,0.113080,2020-06-09,ConfirmedCases
282867,309418,Weston,Wyoming,US,6927,1.130796,2020-06-09,Fatalities
282868,309419,Weston,Wyoming,US,6927,0.113080,2020-06-10,ConfirmedCases


In [25]:
test_attributes = test_data[["County", "Province_State", "Country_Region", "Population", "Weight", "Date"]]

In [26]:
le_test_county = LabelEncoder()
le_test_province_state = LabelEncoder()
le_test_country_region = LabelEncoder()
le_test_date = LabelEncoder()

test_attributes["County_n"] = le_test_county.fit_transform(test_attributes["County"])
test_attributes["Province_State_n"] = le_test_province_state.fit_transform(test_attributes["Province_State"])
test_attributes["Country_Region_n"] = le_test_country_region.fit_transform(test_attributes["Country_Region"])
test_attributes["Date_n"] = le_test_date.fit_transform(test_attributes["Date"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [27]:
test_attributes

,County,Province_State,Country_Region,Population,Weight,Date,County_n,Province_State_n,Country_Region_n,Date_n
0,Autauga,Alabama,US,55869,0.091485,2020-04-27,81,0,0,0
1,Autauga,Alabama,US,55869,0.914848,2020-04-27,81,0,0,0
2,Autauga,Alabama,US,55869,0.091485,2020-04-28,81,0,0,1
3,Autauga,Alabama,US,55869,0.914848,2020-04-28,81,0,0,1
4,Autauga,Alabama,US,55869,0.091485,2020-04-29,81,0,0,2
...,...,...,...,...,...,...,...,...,...,...
282865,Weston,Wyoming,US,6927,1.130796,2020-06-08,1763,50,0,42
282866,Weston,Wyoming,US,6927,0.113080,2020-06-09,1763,50,0,43
282867,Weston,Wyoming,US,6927,1.130796,2020-06-09,1763,50,0,43
282868,Weston,Wyoming,US,6927,0.113080,2020-06-10,1763,50,0,44


In [28]:
test_attributes_n = test_attributes.drop(["County", "Province_State", "Country_Region", "Date"], axis = "columns")
test_attributes_n

,Population,Weight,County_n,Province_State_n,Country_Region_n,Date_n
0,55869,0.091485,81,0,0,0
1,55869,0.914848,81,0,0,0
2,55869,0.091485,81,0,0,1
3,55869,0.914848,81,0,0,1
4,55869,0.091485,81,0,0,2
...,...,...,...,...,...,...
282865,6927,1.130796,1763,50,0,42
282866,6927,0.113080,1763,50,0,43
282867,6927,1.130796,1763,50,0,43
282868,6927,0.113080,1763,50,0,44


In [29]:
test_target_class = test_data[["Target"]]

In [30]:
le_test_target = LabelEncoder()
test_target_class["Target_n"] = le_test_target.fit_transform(test_target_class["Target"])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
test_target_class

,Target,Target_n
0,ConfirmedCases,0
1,Fatalities,1
2,ConfirmedCases,0
3,Fatalities,1
4,ConfirmedCases,0
...,...,...
282865,Fatalities,1
282866,ConfirmedCases,0
282867,Fatalities,1
282868,ConfirmedCases,0


In [32]:
test_result_arr = []
score = 0
tuple_num = len(test_attributes_n)

for i in range(tuple_num):
    result = model.predict([test_attributes_n.loc[i]])[0]
    test_result_arr.append(result)
    if(result == test_target_class.loc[i][1]):
        score = score + 1
score = score / tuple_num

In [33]:
score

1.0

In [34]:
test_attributes["Calculated_Result"] = test_result_arr
test_attributes

,County,Province_State,Country_Region,Population,Weight,Date,County_n,Province_State_n,Country_Region_n,Date_n,Calculated_Result
0,Autauga,Alabama,US,55869,0.091485,2020-04-27,81,0,0,0,0
1,Autauga,Alabama,US,55869,0.914848,2020-04-27,81,0,0,0,1
2,Autauga,Alabama,US,55869,0.091485,2020-04-28,81,0,0,1,0
3,Autauga,Alabama,US,55869,0.914848,2020-04-28,81,0,0,1,1
4,Autauga,Alabama,US,55869,0.091485,2020-04-29,81,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...
282865,Weston,Wyoming,US,6927,1.130796,2020-06-08,1763,50,0,42,1
282866,Weston,Wyoming,US,6927,0.113080,2020-06-09,1763,50,0,43,0
282867,Weston,Wyoming,US,6927,1.130796,2020-06-09,1763,50,0,43,1
282868,Weston,Wyoming,US,6927,0.113080,2020-06-10,1763,50,0,44,0


In [35]:
unique_states = test_attributes.Province_State.unique()

result_table = pd.DataFrame(columns=("Provice_State", "ConfirmedCases", "Fatalities"))

for i in range(len(unique_states)):
    zero_count = len(test_attributes[(test_attributes.Province_State == unique_states[i]) & \
                          (test_attributes.Calculated_Result == 0)])
    one_count = len(test_attributes[(test_attributes.Province_State == unique_states[i]) & \
                          (test_attributes.Calculated_Result == 1)])
    result_table.loc[i] = [unique_states[i], zero_count, one_count]

In [36]:
result_table

,Provice_State,ConfirmedCases,Fatalities
0,Alabama,3015,3015
1,Alaska,1305,1305
2,Arizona,675,675
3,Arkansas,3375,3375
4,California,2610,2610
5,Colorado,2880,2880
6,Connecticut,360,360
7,Delaware,135,135
8,District of Columbia,45,45
9,Florida,3015,3015
